In [1]:
import numpy as np
import torch

In [2]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.0079, -2.3302,  1.0277],
        [ 0.3151,  1.8576,  0.9756]], requires_grad=True)
tensor([-0.3139, -0.0351], requires_grad=True)


In [6]:
def model(x):
    return x @ w.t() + b #@ = matrix multiplication in torch

In [7]:
preds = model(inputs)
print(preds)
print(targets)

tensor([[-112.8236,  189.3837],
        [-140.3185,  254.5548],
        [-253.6443,  332.8922],
        [ -63.2929,  148.0851],
        [-152.6205,  268.3369]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [8]:
#MSE Loss hard coded
def mse(t1,t2): 
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

#MSE FORMULA 
# Matrix 1 - Matrix 2 
# Square the result
# Average

In [9]:
#Compute loss 
loss = mse(preds, targets)
print(loss)

tensor(40139.3633, grad_fn=<DivBackward0>)


In [10]:
#Compute gradient
loss.backward()

In [11]:
#Gradient for weights
print(w)
print(w.grad)

tensor([[-0.0079, -2.3302,  1.0277],
        [ 0.3151,  1.8576,  0.9756]], requires_grad=True)
tensor([[-18244.3711, -21785.7422, -12817.2891],
        [ 12342.8076,  13482.0166,   8223.6963]])


In [12]:
#We use torch.no_grad to indicate to PyTorch that we shouldn't track,
#calculate, or modify gradients while updating the weights and biases.
with torch.no_grad():
    w -= w.grad *1e-5
    b -= b.grad *1e-5
    

In [13]:
loss  = mse(preds, targets)
print(loss)

tensor(40139.3633, grad_fn=<DivBackward0>)


In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [15]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -79.3952,  167.8028],
        [ -96.3393,  226.1940],
        [-201.1426,  299.3169],
        [ -30.5712,  126.6539],
        [-110.1433,  241.1196]], grad_fn=<AddBackward0>)


In [16]:
#LOSS DECRASES 
loss = mse(preds, targets)
print(loss)

tensor(27631.7383, grad_fn=<DivBackward0>)


In [17]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-14788.6562, -18059.7520, -10521.0293],
        [ 10114.5557,  11086.9834,   6745.9761]])
tensor([-179.7183,  120.2174])


In [18]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [19]:
print(w)
print(b)

tensor([[ 0.3224, -1.9318,  1.2611],
        [ 0.0906,  1.6120,  0.8259]], requires_grad=True)
tensor([-0.3099, -0.0378], requires_grad=True)


In [20]:
preds = model(inputs)
print(preds)

tensor([[ -51.9736,  150.0889],
        [ -60.2538,  202.9146],
        [-157.9724,  271.7468],
        [  -3.8265,  109.0725],
        [ -75.2352,  218.7736]], grad_fn=<AddBackward0>)


In [21]:
loss= mse(preds, targets)
print(loss)

tensor(19196.0137, grad_fn=<DivBackward0>)


In [32]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [33]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(92.9820, grad_fn=<DivBackward0>)


In [34]:
print(preds)

tensor([[ 58.9893,  70.3097],
        [ 91.3351,  98.1180],
        [ 94.9991, 138.7619],
        [ 32.7841,  36.8027],
        [110.9094, 114.8135]], grad_fn=<AddBackward0>)


In [35]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression using PyTorch built-ins

In [36]:
import torch.nn as nn

In [37]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [38]:
from torch.utils.data import TensorDataset

In [68]:
#Define Dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [69]:
from torch.utils.data import DataLoader

In [70]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [71]:
for x, y in train_dl:
    print("SET :")
    print(x)
    print(y)
 

SET :
tensor([[ 68.,  97.,  70.],
        [ 69.,  96.,  70.],
        [ 91.,  87.,  65.],
        [102.,  43.,  37.],
        [103.,  43.,  36.]])
tensor([[102., 120.],
        [103., 119.],
        [ 80., 102.],
        [ 22.,  37.],
        [ 20.,  38.]])
SET :
tensor([[ 68.,  96.,  71.],
        [ 88., 134.,  59.],
        [ 91.,  88.,  64.],
        [101.,  44.,  37.],
        [ 73.,  66.,  44.]])
tensor([[104., 118.],
        [118., 132.],
        [ 81., 101.],
        [ 21.,  38.],
        [ 57.,  69.]])
SET :
tensor([[ 92.,  87.,  64.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 74.,  66.,  43.],
        [ 87., 135.,  57.]])
tensor([[ 82., 100.],
        [119., 133.],
        [ 56.,  70.],
        [ 57.,  69.],
        [118., 134.]])


In [72]:
#Define Model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3303,  0.1885,  0.5452],
        [ 0.3791,  0.1617, -0.5482]], requires_grad=True)
Parameter containing:
tensor([ 0.1232, -0.1761], requires_grad=True)


In [73]:
preds = model(inputs)
preds 

tensor([[12.0872, 14.7595],
        [21.5505, 13.4669],
        [28.2715, 22.6803],
        [-5.2862, 25.1597],
        [33.5959,  3.1322],
        [11.5685, 14.9768],
        [21.9072, 12.7569],
        [28.4865, 22.5112],
        [-4.7674, 24.9424],
        [34.4714,  2.2049],
        [12.4440, 14.0496],
        [21.0317, 13.6842],
        [27.9148, 23.3903],
        [-6.1617, 26.0870],
        [34.1147,  2.9149]], grad_fn=<AddmmBackward>)

In [74]:
#loss function
import torch.nn.functional as F

In [75]:
#Define Loss function 
loss_fn = F.mse_loss

In [76]:
loss = loss_fn(preds,targets)
print(loss)

tensor(5559.0127, grad_fn=<MseLossBackward>)


In [77]:
#Define Optimizer 
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

### Utility Function to Train model 

In [79]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    #Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        #Train with batches of data
        for xb, yb in train_dl:
            
            #1. Generate Prediction
            pred = model(xb)
            
            #2. Calculate Loss 
            loss = loss_fn(pred, yb)
            
            #3. Compute gradients
            loss.backward()
            
            #4. Update Parameter using gradients
            opt.step()
            
            #5. Reset the gradients to zero 
            opt.zero_grad()
            
        #print the progress 
        if(epoch+1) % 10 == 0: 
             print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [80]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 511.3353
Epoch [20/100], Loss: 263.4013
Epoch [30/100], Loss: 195.1722
Epoch [40/100], Loss: 21.7504
Epoch [50/100], Loss: 124.6989
Epoch [60/100], Loss: 55.4633
Epoch [70/100], Loss: 48.3152
Epoch [80/100], Loss: 44.5010
Epoch [90/100], Loss: 56.4742
Epoch [100/100], Loss: 68.9780


In [81]:
preds = model(inputs)
print(preds)

tensor([[ 57.4542,  72.3945],
        [ 82.8191,  95.4742],
        [115.3976, 142.1082],
        [ 23.8255,  48.0971],
        [101.5128, 103.5474],
        [ 56.3157,  71.3923],
        [ 82.7918,  94.5090],
        [115.7795, 142.1871],
        [ 24.9640,  49.0993],
        [102.6240, 103.5844],
        [ 57.4269,  71.4293],
        [ 81.6805,  94.4719],
        [115.4250, 143.0734],
        [ 22.7142,  48.0600],
        [102.6513, 104.5496]], grad_fn=<AddmmBackward>)


In [82]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])